In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 6.2.2
    Uninstalling Pillow-6.2.2:
      Successfully uninstalled Pillow-6.2.2


In [ ]:
!pip install flask_cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install werkzeug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fastcore -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade pillow==6.2.2 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-6.2.2-cp38-cp38-manylinux1_x86_64.whl (2.1 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytesseract 0.3.10 requires Pillow>=8.0.0, but you have pillow 6.2.2 which is incompatible.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 6.2.2 which is incompatible.


In [ ]:
# !pip uninstall Pillow

Found existing installation: Pillow 6.2.2
Uninstalling Pillow-6.2.2:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/PIL/*
    /usr/local/lib/python3.8/dist-packages/Pillow-6.2.2.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled Pillow-6.2.2


In [ ]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)


In [ ]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
import torch
from PIL import Image
import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import pandas as pd 
import numpy as np
import os
import torch
from PIL import Image
# import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sentence_transformers import SentenceTransformer, util
from flask import Flask, jsonify, request
from werkzeug.utils import secure_filename
from flask_cors import CORS, cross_origin
import re
from pytesseract import pytesseract

In [ ]:


path_to_tesseract = r"C:\Program Files (x86)\Tesseract-OCR"

modelPath = 'combined_model.pt'

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

mean = [0.4, 0.5, 0.5]
std = [0.4, 0.5, 0.5]


transform = transforms.Compose(
    [transforms.Resize((100, 100)),  
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])

def preprocessor(text):
    text = re.sub(r'[^\w\s]', '',text)
    text = text.lower()
    return text

def pad_tensor(t):
    t = torch.tensor(t)
    padding = 187 - t.size()[0]
    t = torch.nn.functional.pad(t, (0, padding))
    return t

class NN(nn.Module):    
    def __init__(self):
        super(NN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(100*100*3, 256),
            nn.ReLU(inplace=False),
            nn.Linear(256, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class NN_text(nn.Module):    
    def __init__(self):
        super(NN_text, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(187, 256),
            nn.ReLU(inplace=False),
            nn.Linear(256, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class Combined_model(nn.Module):
    def __init__(self, modelA, modelB):
        super(Combined_model, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(6, 128)
        self.classifier1 = nn.Linear(128, 64)
        self.classifier2 = nn.Linear(64, 32)
        self.classifier3 = nn.Linear(32, 16)
        
        self.out1 = nn.Linear(16, 2,)
        self.out2 = nn.Linear(16, 2)
        self.out3 = nn.Linear(16, 2)
        self.out4 = nn.Linear(16, 2)
        self.out5 = nn.Linear(16, 3)

        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        
        x = self.classifier(x)
        x = self.classifier1(x)
        x = self.classifier2(x)
        x = self.classifier3(x)
        
        out1 = self.out1(x)
        out2 = self.out2(x)
        out3 = self.out3(x)
        out4 = self.out4(x)
        out5 = self.out5(x)
        return out1, out2, out3, out4, out5


net = NN()
net_text = NN_text()
combined_model = Combined_model(net, net_text)

combined_model.load_state_dict(torch.load(modelPath))
combined_model.eval()

app = Flask(__name__)
cors = CORS(app)


app.secret_key = "secret key" 
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024 
path = os.getcwd()
UPLOAD_FOLDER = os.path.join(path, 'uploads')

if not os.path.isdir(UPLOAD_FOLDER):
    os.mkdir(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif','docx'])




def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


diction = dict()

@app.route('/send_pdf', methods=['POST'])
def upload_pdf():
    if request.method == 'POST':

        if 'file' not in request.files:
            print('No file part')

        file = request.files['file']
        if file.filename == '':
            print('No file selected for uploading')
            
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))      
            print('File successfully uploaded')
            savedImage = Image.open(os.path.join(app.config['UPLOAD_FOLDER'], filename)).convert('RGB')
            pytesseract.tesseract_cmd = path_to_tesseract
  
# Passing the image object to image_to_string() function
# This function will extract the text from the image
            text = pytesseract.image_to_string(savedImage)
            
            # Displaying the extracted text
            savedText = preprocessor(text[:-1])
            paddedText = pad_tensor(sentence_model.encode(savedText, convert_to_tensor=True))
            paddedText = paddedText.unsqueeze(dim=0)
            transImg = transform(savedImage)
            transImg = transImg.unsqueeze(dim=0)

            label1, label2, label3, label4, label5 = combined_model(transImg, paddedText)

            diction = {
                'humour':int(label1.argmax(1).item()),
                'sarcasm':int(label2.argmax(1).item()),
                'offensive':int(label3.argmax(1).item()),
                'motivational':int(label4.argmax(1).item()),
                'overall_sentiment':int(label5.argmax(1).item())

            }
            
            
        else:
            print('File format not allowed')


    return diction

if __name__ == "__main__":
    app.run(host='127.0.0.1',port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
